In [3]:
import sys
sys.path.append('..')
from CompoundPoisson2 import CompoundPoisson2
from MCABS import MCLossAbs
import numpy as np
import scipy.stats as st
import math
from scipy.optimize import minimize
import time

In [2]:
#class for loss function of the following form :  l(x) = 1 / (1 + alpha)(sum(exp(beta * xi)) + alpha*exp(beta *sum(xi)))- (alpha+d)/(alpha+1)
class MCLossFunction1(MCLossAbs):
    def __init__(self, X, alpha, beta = 1):
        self.__alpha = alpha
        self.__beta = beta
        super(MCLossFunction1, self).__init__(X)
        
    def shortfall_risk(self, m = None):
        m = self._check_argument(m)
        beta, alpha = self.__beta, self.__alpha
        #This substract from the col i of X, m_i
        #X is a matrix of dim columns and N rows where N is the sample's length
        x_minus_m = np.subtract(self.X, m)
        exp_x_minus_m = np.exp(beta * x_minus_m)
        #axis = 1 means we sum the elements of each row
        mean_sum1 = np.mean(np.sum(exp_x_minus_m, axis = 1))
        mean_sum2 = alpha * np.mean(np.exp(beta * np.sum(x_minus_m, axis = 1)))
        return (1 / (1 + alpha)) * (mean_sum1 + mean_sum2) - (alpha + self.dim) / (alpha + 1)
    
    def shortfall_risk_jac(self, m):
        m = self._check_argument(m)
        alpha, beta = self.__alpha, self.__beta
        x_minus_m = np.subtract(self.X, m)
        common_mean = alpha * np.mean(np.exp(beta * np.sum(x_minus_m, axis = 1)))
        ind_mean = np.mean(np.exp(beta * x_minus_m), axis = 0)
        return beta / (1 + alpha) * (ind_mean + common_mean)

NameError: name 'MCLossAbs' is not defined

In [ ]:
#Case: rho = 0, alpha = 1, beta = 1

#M is the length of the sample
np.random.seed(seed=0)
M = 100000
corr = 0.9
mu = [0., 0.]
sigma = [[1., corr], [corr, 1.]]

rv = st.multivariate_normal(mean = mu, cov = sigma, allow_singular = True)
distr = rv.rvs(size = M)

alpha, beta = 1., 1.

maxiter = 3500

start_time = time.time()

loss = MCLossFunction1(distr, alpha, beta)


init = np.ones(loss.dim)
cons = ({'type': 'ineq',
         'fun' : lambda x: loss.ineq_constraint(x),
         'jac' : lambda x: loss.ineq_constraint_jac(x)})


res = minimize(loss.objective, init, 
              jac = loss.objective_jac, 
              constraints = cons, 
               method='SLSQP',
               options={'maxiter': maxiter, 'disp': True})
print(res)
print("--- MC Method: %s seconds ---" % (time.time() - start_time))